# Lab 1: Approximating natural languages

## 1  Preparation

In [1]:
def load_file(fname):
    with open(fname) as f:
        return f.read()
    
    
hamlet = load_file("corpus/norm_hamlet.txt")
romeo = load_file("corpus/norm_romeo_and_juliet.txt")
wiki = load_file("corpus/norm_wiki_sample.txt")

corpus = [hamlet, romeo, wiki]

## 2  Zeroth-order approximation

In [2]:
# Definicja 27 zankowego alfabetu.
# 'a' do 'z' + ' '
alphabet = [chr(ord('a') + i) for i in range(26)]
alphabet.append(" ")

# Długość generowanego textu
text_len = 1000000

In [13]:
def avg_word_len(text):
    words = text.split(" ")
    return sum([len(w) for w in words]) / len(words)

In [3]:
import random

zeroth_order_text = "".join(random.choices(alphabet, k=text_len))
zeroth_avg_word_len = avg_word_len(zeroth_order_text)

print("Średnia długość wyrazu", zeroth_avg_word_len)

Średnia długość wyrazu 25.98038527951651


## 3  Frequency of letters

In [4]:
from collections import Counter
import matplotlib.pyplot as plt

def probability(text):
    counter = Counter(text)
    return dict([(token, count / len(text)) for token, count in counter.items()])

merged_text = "".join(corpus)
letters_prob = probability(merged_text)
most_common_letters = sorted(letters_prob.items(), key=lambda x: x[1], reverse=True)

plt.bar(*zip(*most_common_letters))

<BarContainer object of 37 artists>

## 4  First-order approximation

In [5]:
first_order_text = "".join(random.choices(list(letters_prob.keys()), weights=list(letters_prob.values()), k=text_len))
first_avg_word_len = avg_word_len(first_order_text)

print("Średnia długość wyrazu:", first_avg_word_len)

Średnia długość wyrazu: 4.856864238022725


## 5  Conditional probability of letters

In [12]:
def get_seq(text, seq_len):
    return [text[i:i+seq_len] for i in range(len(text) - seq_len)]

In [11]:
two_most_common_letters = [l for l, p in most_common_letters[:2]]

print("Dwa najczęsciej pojawiające się znaki:", two_most_common_letters)


seqs = get_seq(merged_text, 2)
seqs_prob = probability(seqs)

print(dict())



# def sequences_prob(text, letters_p, seq_len):
#     seqs = [text[i:i + seq_len for i in range(len(text) - lookback)]
#     seqs_prob = probability(seqs)
#     return seqs_prob


# seq_2_prob = sequences_prob(merged_text, letters_prob, 1)

    

# # P(i,j)
# letter_pairs_prob = probability(letter_pairs)

# # P(j|i) = P(i,j) / P(i)
# condtional_prob = {f"{l[1]}|{l[0]}": p / letters_prob[l[0]] for l, p in letter_pairs_prob.items()}
# condtional_prob

# # for letter, prob in most_common_letters[:2]:
# #     print([(l,p) for l, p in condtional_prob.items() if l == letter])

# # print(list(zip(letter_pairs, letter_pairs_freq)))

Dwa najczęsciej pojawiające się znaki: [' ', 'e']
{}


## 6  Approximations based on Markov sources